# SageMaker Model Monitor - visualização dos resultados do monitoramento


O contêiner pré-construído do SageMaker calcula uma variedade de estatísticas e avalia as restrições fora da caixa. Este notebook demonstra como você pode visualizá-los. Você pode pegar o arn ProcessingJob das execuções por trás de um MonitoringSchedule e usar este bloco de notas para visualizar os resultados.

Vamos importar algumas bibliotecas Python que serão úteis para visualização

In [1]:
!pip install numpy==1.16.5

/opt/conda/lib/python3.7/site-packages/secretstorage/dhcrypto.py:16: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/lib/python3.7/site-packages/secretstorage/util.py:25: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
     |████████████████████████████████| 17.3 MB 4.5 MB/s eta 0:00:01
  Attempting uninstall: numpy
    Found existing installation: numpy 1.20.3
    Uninstalling numpy-1.20.3:
      Successfully uninstalled numpy-1.20.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pyarrow 5.0.0 requires numpy>=1.16.6, but you have numpy 1.16.5 which is incompatible.
numba 0.54.0 requires numpy<1.21,>=1.17, but you have numpy 1.16.5 which is incompatible.


In [8]:
from IPython.display import HTML, display
import json
import os
import boto3

import sagemaker
from sagemaker import session
from sagemaker.model_monitor import MonitoringExecution
from sagemaker.s3 import S3Downloader

# As funções para plotar e renderizar estatísticas de distribuição ou violações de restrição são implementadas em um arquivo `utils`, então vamos carregá-lo.
import utils as mu

import numpy as np
np.version.version

'1.16.5'

## Obtenção de detalhes de execução e baseline (linha de base).

Insira o arn ProcessingJob para uma execução de um MonitoringSchedule abaixo para obter os arquivos de resultado associados a essa execução

A cada hora, um novo Job será gerado.

In [15]:
processing_job_arn = "arn:aws:sagemaker:us-east-2:325011675573:processing-job/model-monitoring-202109122300-c2649fb5f062697d54a33206"

In [13]:
execution = MonitoringExecution.from_processing_arn(
    sagemaker_session=session.Session(), processing_job_arn=processing_job_arn
)
exec_inputs = {inp["InputName"]: inp for inp in execution.describe()["ProcessingInputs"]}
exec_results = execution.output.destination

In [ ]:
baseline_statistics_filepath = (
    exec_inputs["baseline"]["S3Input"]["S3Uri"] if "baseline" in exec_inputs else None
)
execution_statistics_filepath = os.path.join(exec_results, "statistics.json")
violations_filepath = os.path.join(exec_results, "constraint_violations.json")

baseline_statistics = (
    json.loads(S3Downloader.read_file(baseline_statistics_filepath))
    if baseline_statistics_filepath is not None
    else None
)
execution_statistics = json.loads(S3Downloader.read_file(execution_statistics_filepath))
violations = json.loads(S3Downloader.read_file(violations_filepath))["violations"]

In [14]:
execution_statistics_filepath

's3://sagemaker-us-east-2-325011675573/sagemaker/DEMO-ModelMonitor/reports/DEMO-xgb-churn-pred-model-monitor-2021-09-12-21-35-18/DEMO-xgb-churn-pred-model-monitor-schedule-2021-09-12-22-57-21/2021/09/12/23/statistics.json'

## Visão geral

O código abaixo mostra as violações e verificações de restrição em todos os recursos em uma tabela simples.

In [9]:
mu.show_violation_df(
    baseline_statistics=baseline_statistics,
    latest_statistics=execution_statistics,
    violations=violations,
)

,data_type,completeness,baseline_drift,categorical_values
Account Length,Integral,100.00%,N/A,N/A
Area Code_408,Fractional,100.00%,N/A,N/A
Area Code_415,Fractional,100.00%,N/A,N/A
Area Code_510,Fractional,100.00%,N/A,N/A
Churn,Fractional,100.00%,N/A,N/A
CustServ Calls,Integral,100.00%,N/A,N/A
Day Calls,Integral,100.00%,N/A,N/A
Day Mins,Fractional,100.00%,N/A,N/A
Eve Calls,Integral,100.00%,N/A,N/A
Eve Mins,Fractional,100.00%,N/A,N/A


## Distribuições

Esta seção visualiza a distribuição e renderiza as estatísticas de distribuição para todos os recursos

In [10]:
features = mu.get_features(execution_statistics)
feature_baselines = mu.get_features(baseline_statistics)

In [ ]:
mu.show_distributions(features)

### Estatísticas de execução vs baseline

In [ ]:
mu.show_distributions(features, feature_baselines)